In [4]:
!pip install pygame

Defaulting to user installation because normal site-packages is not writeable


In [15]:
import pygame
import random
import time

# Initialize pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 800, 600

# Colors (RGB format)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)

# Initialize screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Snake Game")

# Clock for controlling game speed
clock = pygame.time.Clock()

# Snake block size
BLOCK_SIZE = 10

# Font for displaying score and text
font = pygame.font.SysFont("bahnschrift", 25)
font_large = pygame.font.SysFont("bahnschrift", 40)

# Function to display score
def show_score(score):
    value = font.render(f"Your Score: {score}", True, WHITE)
    screen.blit(value, [10, 10])

# Function to display text
def display_text(text, color, y_offset=0):
    value = font_large.render(text, True, color)
    screen.blit(value, [WIDTH//3, HEIGHT//4 + y_offset])

# Function to draw buttons
def draw_button(text, x, y, width, height, color, text_color):
    pygame.draw.rect(screen, color, [x, y, width, height])
    value = font.render(text, True, text_color)
    screen.blit(value, [x + width//4, y + height//4])

# Function to choose the difficulty level, snake color, and start the game
def choose_settings():
    snake_color = GREEN
    speed = 3  # Default medium speed
    level = "Medium"  # Default level
    
    # Settings Menu Loop
    settings_selected = False
    while not settings_selected:
        screen.fill(BLACK)

        # Difficulty Level selection
        display_text("Select Difficulty:", WHITE, 0)
        draw_button("Easy", WIDTH//4, HEIGHT//2 - 90, 150, 50, GREEN, WHITE)
        draw_button("Medium", WIDTH//4 + 160, HEIGHT//2 - 90, 150, 50, YELLOW, BLACK)
        draw_button("Hard", WIDTH//4 + 320, HEIGHT//2 - 90, 150, 50, RED, WHITE)

        # Snake color selection
        display_text("Select Snake Color:", WHITE, 220)
        #draw_button("Green", WIDTH//4, HEIGHT//2 - 30, 150, 50, GREEN, WHITE)
        draw_button("Red", WIDTH//4 + 160, HEIGHT//2 - 30, 150, 50, RED, WHITE)
        draw_button("Blue", WIDTH//4 + 320, HEIGHT//2 - 30, 150, 50, BLUE, WHITE)
        draw_button("Yellow", WIDTH//4, HEIGHT//2 - 30, 150, 50, YELLOW, BLACK)

        # Start Button
        draw_button("Start", WIDTH//4 + 160, HEIGHT//2 + 120, 150, 50, BLUE, WHITE)

        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()

            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_x, mouse_y = pygame.mouse.get_pos()

                # Difficulty Level Selection
                if WIDTH//4 <= mouse_x <= WIDTH//4 + 150:
                    if HEIGHT//2 - 90 <= mouse_y <= HEIGHT//2 - 40:
                        level = "Easy"
                        speed = 5
                elif WIDTH//4 + 160 <= mouse_x <= WIDTH//4 + 310:
                    if HEIGHT//2 - 90 <= mouse_y <= HEIGHT//2 - 40:
                        level = "Medium"
                        speed = 3
                elif WIDTH//4 + 320 <= mouse_x <= WIDTH//4 + 470:
                    if HEIGHT//2 - 90 <= mouse_y <= HEIGHT//2 - 40:
                        level = "Hard"
                        speed = 1

                # Snake Color Selection
                if WIDTH//4 <= mouse_x <= WIDTH//4 + 150:
                    if HEIGHT//2 - 30 <= mouse_y <= HEIGHT//2 + 20:
                        snake_color = GREEN
                    elif HEIGHT//2 + 50 <= mouse_y <= HEIGHT//2 + 100:
                        snake_color = YELLOW
                elif WIDTH//4 + 160 <= mouse_x <= WIDTH//4 + 310:
                    if HEIGHT//2 - 30 <= mouse_y <= HEIGHT//2 + 20:
                        snake_color = RED
                elif WIDTH//4 + 320 <= mouse_x <= WIDTH//4 + 470:
                    if HEIGHT//2 - 30 <= mouse_y <= HEIGHT//2 + 20:
                        snake_color = BLUE

                # Start the game
                if WIDTH//4 + 160 <= mouse_x <= WIDTH//4 + 310:
                    if HEIGHT//2 + 120 <= mouse_y <= HEIGHT//2 + 170:
                        settings_selected = True
                        return snake_color, speed

# Snake movement and growth logic
def draw_snake(snake_list, snake_color):
    for block in snake_list:
        pygame.draw.rect(screen, snake_color, [block[0], block[1], BLOCK_SIZE, BLOCK_SIZE])

# Main function for the game
def game_loop(snake_color, SNAKE_SPEED):
    # Initial position of the snake
    snake_pos = [100, 50]
    snake_body = [[100, 50], [90, 50], [80, 50]]

    # Initial direction and score
    direction = 'RIGHT'
    change_to = direction
    score = 0

    # Position of the first food
    food_pos = [random.randrange(1, (WIDTH//BLOCK_SIZE)) * BLOCK_SIZE,
                random.randrange(1, (HEIGHT//BLOCK_SIZE)) * BLOCK_SIZE]
    food_spawn = True

    game_over = False

    while not game_over:
        # Event handling for key presses
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP and direction != 'DOWN':
                    change_to = 'UP'
                elif event.key == pygame.K_DOWN and direction != 'UP':
                    change_to = 'DOWN'
                elif event.key == pygame.K_LEFT and direction != 'RIGHT':
                    change_to = 'LEFT'
                elif event.key == pygame.K_RIGHT and direction != 'LEFT':
                    change_to = 'RIGHT'
        
        # If the snake hits the boundaries, game over
        if (snake_pos[0] < 0 or snake_pos[0] >= WIDTH or
            snake_pos[1] < 0 or snake_pos[1] >= HEIGHT):
            game_over = True
        
        # Update direction based on user input
        direction = change_to
        if direction == 'UP':
            snake_pos[1] -= BLOCK_SIZE
        elif direction == 'DOWN':
            snake_pos[1] += BLOCK_SIZE
        elif direction == 'LEFT':
            snake_pos[0] -= BLOCK_SIZE
        elif direction == 'RIGHT':
            snake_pos[0] += BLOCK_SIZE
        
        # Snake body growing mechanism
        # Add new block to the head
        snake_body.insert(0, list(snake_pos))
        if snake_pos == food_pos:
            score += 1
            food_spawn = False
        else:
            # Remove last block of the snake
            snake_body.pop()

        # Spawn food when eaten
        if not food_spawn:
            food_pos = [random.randrange(1, (WIDTH//BLOCK_SIZE)) * BLOCK_SIZE,
                        random.randrange(1, (HEIGHT//BLOCK_SIZE)) * BLOCK_SIZE]
        food_spawn = True

        # Fill screen and draw objects
        screen.fill(BLACK)
        show_score(score)
        draw_snake(snake_body, snake_color)
        pygame.draw.rect(screen, RED, [food_pos[0], food_pos[1], BLOCK_SIZE, BLOCK_SIZE])

        # Collision with itself
        for block in snake_body[1:]:
            if snake_pos == block:
                game_over = True

        # Update the screen
        pygame.display.update()

        # Control the speed of the snake
        clock.tick(SNAKE_SPEED)

    # Game over message
    screen.fill(BLACK)
    game_over_text = font.render("Game Over! Press any key to exit.", True, RED)
    screen.blit(game_over_text, [WIDTH//6, HEIGHT//3])
    show_score(score)
    pygame.display.update()
    time.sleep(2)

    # Exit game
    pygame.quit()

# Start the game setup
snake_color, SNAKE_SPEED = choose_settings()

# Run the game
game_loop(snake_color, SNAKE_SPEED)
